# AI Learns To Play Connect 4


## Part 1: Recording Random Games

If you are brand new to this series, check out Part 1 - Introduction. We are building an AI using Machine Learning to play 4 In A Row.

The starter code provided is a great start. We only need to make 2 quick changes 

1. Add a filename which we will save the data to
2. Modify the make_move function which is called whenever the bot is asked which column it would like to put a token in.


### 1. Create a filepath to save games to.

Each time the bot starts a new game, the main function creates an object called State. I decided to piggy back off of this state to hold the name of the new file. Since I expect to record thousands/hundreds of thousands of games I thought it would be easiest to name them the current time so we don't get any duplicates. Here is the added line I have in the State Class. 

I have added a folder called "Saved_Games" for all the files to be stored in and imported the time module

In [1]:
import time

# Dummy Settings Class
class Settings():
    None
    
class Field():
    None
    
class State(object):
    def __init__(self):
        self.settings = Settings()
        self.field = Field()
        self.round = 0
        
        self.filename = './Saved_Games/' + str(int(time.time())) + '.npy'

In [2]:
state = State()

# Print Results
print(state.filename)

./Saved_Games/1531347773.npy


### 2. Looking at the make_move function.

Firstly lets look at the original function.

Basically, everytime the game asks the bot for a move it will randomly select an integer between 0 and 6. (0 is actually the first column, and 6 is the 7th column). This is also how array indexing working in python so it is nice that it works like this.

In [3]:
import random

def make_move(state):

    # TODO: Implement bot logic here
    rand_col = random.randint(0, state.settings.field_width-1)

    return 'place_disc {}'.format(rand_col)

state.settings.field_width = 7 # The game would normally set this value, we will manually set it for this notebook

# Print Results
for i in range(5):
    print('Move {}: {}'.format(i, make_move(state)))

Move 0: place_disc 1
Move 1: place_disc 3
Move 2: place_disc 0
Move 3: place_disc 5
Move 4: place_disc 5


### 3. Modifying the make_move function

So we have 2 goals at this stage.

1. We want to record the game history so we can start building a neural network to make some predictions
2. The current progmam sometimes choses an invalid column (One that is already full) which makes the bot instantly lose. This may make our life a bit tricky down the track

##### Storing the games:

The field state can be assessed through the state object. This is the layout of the board. It is originally a list, but we can convert it to a numpy array quite easily with np.array()

We want to store the entire history of the game, not just the final position. To do this we will setup a numpy array of size (Rounds, Height, Width). For each round, we have a 2D array which is a record of the board at that round

- If this is a brand new game, we want to make a new array to store all the information. If not, we want to load the current history array. I am saving the array to the disk after each move to simplify the implementation, I know this probably isn't the fastest way!

Now we itterate over all the rows.
- Yellow tokens are stored with a 1's
- Red tokens are stored as 0's
- Since our numpy array is already zero's we want to make the red tokens a negative 1.
- So for each row, we set yellow and red tokens to either a 1 (Yellow) or -1 (Red)

##### Make sure the bot always chooses a valid column
- We search row 0 of the current board for any values that are zero (empty)
- We then shuffle the valid indicies before choosing the first one as our final choice

Here is the final function

In [4]:
import numpy as np

def make_move(state):
    
    field_state = np.array(state.field.field_state)
    
    if state.round == 0:
        current_board = np.zeros((30,6, 7))
    else:
        current_board = np.load(state.name)

    for row in range(6):
        yellow_idx = np.where(field_state[row] == '1')[0]
        red_idx = np.where(field_state[row] == '0')[0]

        current_board[state.round][row][yellow_idx] = 1
        current_board[state.round][row][red_idx] = -1

    valid_idx = np.where(current_board[state.round][0] == 0)[0]
    np.random.shuffle(valid_idx)

    rand_col = valid_idx[0]

    np.save(state.name, current_board)
    return 'place_disc {}'.format(rand_col)


In Part 2 we will read the game files and see what we can do with the data

Here is the complete code if you'd prefer

In [5]:
#!/usr/bin/env python3
import sys
import numpy as np
import time

class Settings(object):
    def __init__(self):
        self.timebank = None
        self.time_per_move = None
        self.player_names = None
        self.your_bot = None
        self.your_botid = None
        self.field_width = None
        self.field_height = None


class Field(object):
    def __init__(self):
        self.field_state = None

    def update_field(self, celltypes, settings):
        self.field_state = [[] for _ in range(settings.field_height)]
        n_cols = settings.field_width
        for idx, cell in enumerate(celltypes):
            row_idx = idx // n_cols
            self.field_state[row_idx].append(cell)


class State(object):
    def __init__(self):
        self.settings = Settings()
        self.field = Field()
        self.round = 0

        self.name = './Saved_Games/' + str(int(time.time())) + '.npy'

def parse_communication(text):
    """ Return the first word of the communication - that's the command """
    return text.strip().split()[0]


def settings(text, state):
    """ Handle communication intended to update game settings """
    tokens = text.strip().split()[1:] # Ignore token 0, it's the string "settings".
    cmd = tokens[0]
    if cmd in ('timebank', 'time_per_move', 'your_botid', 'field_height', 'field_width'):
        # Handle setting integer settings.
        setattr(state.settings, cmd, int(tokens[1]))
    elif cmd in ('your_bot',):
        # Handle setting string settings.
        setattr(state.settings, cmd, tokens[1])
    elif cmd in ('player_names',):
        # Handle setting lists of strings.
        setattr(state.settings, cmd, tokens[1:])
    else:
        raise NotImplementedError('Settings command "{}" not recognized'.format(text))


def update(text, state):
    """ Handle communication intended to update the game """
    tokens = text.strip().split()[2:] # Ignore tokens 0 and 1, those are "update" and "game" respectively.
    cmd = tokens[0]
    if cmd in ('round',):
        # Handle setting integer settings.
        setattr(state.settings, 'round', int(tokens[1]))
    if cmd in ('field',):
        # Handle setting the game board.
        celltypes = tokens[1].split(',')
        state.field.update_field(celltypes, state.settings)


def action(text, state):
    """ Handle communication intended to prompt the bot to take an action """
    tokens = text.strip().split()[1:] # Ignore token 0, it's the string "action".
    cmd = tokens[0]
    if cmd in ('move',):
        move = make_move(state)
        state.round += 1
        return move
    else:
        raise NotImplementedError('Action command "{}" not recognized'.format(text))

def make_move(state):

    # TODO: Implement bot logic here

    if state.round == 0:
        current_board = np.zeros((30,6, 7))
    else:
        current_board = np.load(state.name)

    field_state = np.array(state.field.field_state)

    for row in range(6):
        yellow_idx = np.where(field_state[row] == '1')[0]
        red_idx = np.where(field_state[row] == '0')[0]

        current_board[state.round][row][yellow_idx] = 1
        current_board[state.round][row][red_idx] = -1

    valid_idx = np.where(current_board[state.round][0] == 0)[0]
    np.random.shuffle(valid_idx)

    rand_col = valid_idx[0]

    np.save(state.name, current_board)
    return 'place_disc {}'.format(rand_col)

def main():
    command_lookup = { 'settings': settings, 'update': update, 'action': action }
    state = State()
    for input_msg in sys.stdin:
        cmd_type = parse_communication(input_msg)
        command = command_lookup[cmd_type]

        # Call the correct command.
        res = command(input_msg, state)

        # Assume if the command generates a string as output, that we need
        # to "respond" by printing it to stdout.
        if isinstance(res, str):
            print(res)
            sys.stdout.flush()



if __name__ == '__main__':
    main()
